<a href="https://colab.research.google.com/github/tranly-dev/ustomer-churn-prediction./blob/main/ustomer-churn-prediction./notebooks/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Customer Churn Prediction – Model Training

Trong notebook này, ta sẽ xây dựng mô hình **dự đoán khả năng khách hàng rời bỏ (churn)** dựa trên dữ liệu `Churn_Modelling.csv`.

## 🎯 Mục tiêu:
- Hiểu và chuẩn bị dữ liệu đầu vào
- Xây dựng mô hình Logistic Regression (baseline)
- Đánh giá độ chính xác (accuracy, AUC, confusion matrix)
- So sánh với mô hình khác (RandomForest)


## ⚙️ 1. Import thư viện cần thiết


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, classification_report, roc_auc_score,
                             RocCurveDisplay, ConfusionMatrixDisplay)
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42
pd.set_option('display.max_columns', 100)


## 📥 2. Nạp dữ liệu


In [ ]:
# Thay đường dẫn tương ứng repo của bạn nếu khác
csv_path = "./data/raw/Churn_Modelling.csv"

df = pd.read_csv(csv_path)
df.head()


## 🧹 3. Tiền xử lý dữ liệu
- Loại bỏ cột không cần thiết (`RowNumber`, `CustomerId`, `Surname`)
- Xác định biến mục tiêu (`Exited`)
- Tách dữ liệu train/test


In [ ]:
# Cột target
TARGET_COL = "Exited"

# Bỏ ID/metadata không mang thông tin dự báo
drop_cols = ["RowNumber", "CustomerId", "Surname"]
df_model = df.drop(columns=drop_cols, errors="ignore").copy()

# Kiểm tra NA (nếu có thì xử lý; ở dataset này thường không có)
na_counts = df_model.isna().sum()
na_counts[na_counts>0]


In [ ]:
# Phân loại cột số & cột hạng mục (categorical)
numeric_features = df_model.drop(columns=[TARGET_COL]).select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_model.drop(columns=[TARGET_COL]).select_dtypes(exclude=[np.number]).columns.tolist()

numeric_features, categorical_features


In [ ]:
# Tách X, y và train/test
X = df_model.drop(columns=[TARGET_COL])
y = df_model[TARGET_COL].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


## 🏗 4. Xây dựng pipeline tiền xử lý và Logistic Regression


In [ ]:
# Bộ biến đổi theo cột
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# Mô hình baseline
log_reg = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=None # logistic không dùng n_jobs
)

# Pipeline kết hợp
pipe_lr = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", log_reg)
])

pipe_lr


## 📈 5. Đánh giá mô hình Logistic Regression


In [ ]:
pipe_lr.fit(X_train, y_train)

pred_test = pipe_lr.predict(X_test)
proba_test = pipe_lr.predict_proba(X_test)[:, 1]

print("AUC:", roc_auc_score(y_test, proba_test))
print(classification_report(y_test, pred_test, digits=4))

fig, ax = plt.subplots(1,2, figsize=(12,5))

ConfusionMatrixDisplay.from_predictions(y_test, pred_test, ax=ax[0], cmap="Blues")
ax[0].set_title("Confusion Matrix")

RocCurveDisplay.from_predictions(y_test, proba_test, ax=ax[1])
ax[1].set_title("ROC Curve")

plt.show()


## 🌲 6. So sánh với mô hình RandomForest


In [ ]:
cv_auc = cross_val_score(pipe_lr, X, y, cv=5, scoring="roc_auc", n_jobs=-1)
print("CV AUC:", cv_auc.mean(), "+/-", cv_auc.std())


## 💾 7. Lưu mô hình tốt nhất


In [ ]:
param_grid = {
    "model__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    "model__penalty": ["l2"],            # liblinear/saga với l1; ở đây dùng mặc định solver='lbfgs'
    "model__fit_intercept": [True, False]
}

grid = GridSearchCV(
    estimator=pipe_lr,
    param_grid=param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best CV AUC:", grid.best_score_)

best_model = grid.best_estimator_

pred_test = best_model.predict(X_test)
proba_test = best_model.predict_proba(X_test)[:, 1]
print("TEST AUC:", roc_auc_score(y_test, proba_test))
print(classification_report(y_test, pred_test, digits=4))


## 🧪 8. Thử dự đoán một khách hàng mẫu


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=300,
        random_state=RANDOM_STATE,
        class_weight="balanced_subsample",
        n_jobs=-1
    ))
])

rf.fit(X_train, y_train)
proba_rf = rf.predict_proba(X_test)[:, 1]
pred_rf  = rf.predict(X_test)

print("RF AUC:", roc_auc_score(y_test, proba_rf))
print(classification_report(y_test, pred_rf, digits=4))


## 💾 9. Lưu mô hình tốt nhất (Save the trained model)

Sau khi đánh giá hiệu năng các mô hình, ta chọn mô hình có hiệu suất cao nhất (ở đây là **RandomForest**)  
và lưu lại thành file `.pkl` để sử dụng lại cho các bước **triển khai (deployment)** hoặc **suy luận (inference)** sau này.




In [ ]:
import joblib
joblib.dump(best_model, "./data/processed/model_churn_lr.pkl")
print("Saved to ./data/processed/model_churn_lr.pkl")c


## 🧪 10. Dự đoán thử trên một khách hàng (Inference Demo)

Sau khi đã huấn luyện và lưu mô hình, ta có thể **tải lại** mô hình đó để kiểm thử dự đoán cho **một khách hàng cụ thể**.

> ✅ Các bước thực hiện:
> 1. Lấy một hàng dữ liệu mẫu từ `X_test`  
> 2. Dự đoán xác suất khách hàng sẽ rời bỏ (`churn probability`)  
> 3. Phân loại khách hàng theo ngưỡng xác suất (`threshold = 0.5`)



In [ ]:
# Ví dụ lấy 1 hàng từ test set
sample = X_test.iloc[[0]].copy()
sample, y_test.iloc[0]


In [ ]:
def predict_one(model, row_df, thr=0.5):
    proba = model.predict_proba(row_df)[:, 1][0]
    pred  = int(proba >= thr)
    return {"proba": float(proba), "pred": pred}

predict_one(best_model, sample, thr=0.5)
